In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from torchvision.models import resnet50, ResNet50_Weights
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader
import torch

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
from copy import deepcopy

In [ ]:
def create_df(base_dir, mp):
    x = []
    y = []
    for i in os.listdir(base_dir):
        label_path = os.path.join(base_dir, i)
        for j in os.listdir(label_path):
            im_path = os.path.join(label_path, j)
            original = Image.open(im_path).convert("RGB")
            x += [np.array(original)]
            y += [mp[i]]
            if i == "Reject":
                flipper = original.transpose(method=Image.FLIP_LEFT_RIGHT)
                x += [np.array(original)]
                y += [mp[i]]
                
    return x, y

In [ ]:
EPOCHS = 30
IM_SIZE = 224
LR = 0.1
GAMMA = 0.1
STEP = 10
BATCH = 32
NUM_CLASSES = 2

In [ ]:
base_dir = "/kaggle/input/tomatofruits/Two Classes"
class_index = {"Healthy": 1, "Reject": 0}
index_class = {1: "Healthy", 0: "Reject"}

In [ ]:
x, y = create_df(base_dir, class_index)

In [ ]:
x_train, x_testing, y_train, y_testing = train_test_split(x, y, random_state=42, test_size=0.2)
x_val, x_test, y_val, y_test = train_test_split(x_testing, y_testing, random_state=42, test_size=0.5)

In [ ]:
class Clod(Dataset):
    def __init__(self, data, target, transform):
        super(Clod, self).__init__()
        self.data = data
        self.target = target
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, x):
        image, label = self.data[x], self.target[x]
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

In [ ]:
transform = transforms.Compose([transforms.ToPILImage(),
                              transforms.ToTensor(),
                              transforms.Resize((IM_SIZE, IM_SIZE)),
                              transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225])])

In [ ]:
train_ds = Clod(x, y, transform)
val_ds = Clod(x, y, transform)

train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH, shuffle=False)

In [ ]:
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
for param in model.parameters():
    param.require_grad_ = False
    
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, NUM_CLASSES)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP, gamma=GAMMA)

In [ ]:
best_model = deepcopy(model)
best_acc = 0

acc_train = []
acc_val = []
loss_train = []
loss_val = []

for i in range(1, EPOCHS+1):
    model.train()
    
    train_loss = 0
    train_acc = 0
    train_total = 0
    
    for data, label in train_dl:
        optimizer.zero_grad()
        if torch.cuda.is_available():
            data, label = data.cuda(), label.cuda()
            
        out = model(data)
        loss = criterion(out, label)
        train_loss += loss.item()
        train_total += out.size(0)
        train_acc += (out.argmax(1) == label).sum().item()
        loss.backward()
        optimizer.step()
    train_loss /= train_total
    train_acc /= train_total
    model.eval()
    
    val_loss = 0
    val_total = 0
    val_acc = 0
    
    with torch.no_grad():
        for data, label in val_dl:
            if torch.cuda.is_available():
                data, label = data.cuda(), label.cuda()
                
            out = model(data)
            loss = criterion(out, label)
            val_loss += loss.item()
            val_acc += (out.argmax(1) == label).sum().item()
            val_total += out.size(0)
            
    val_loss /= val_total
    val_acc /= val_total
    
    acc_train += [train_acc]
    acc_val += [val_acc]
    loss_train += [train_loss]
    loss_val += [train_acc]
    
    if val_acc > best_acc:
        best_model = deepcopy(model)
        best_acc = val_acc
        
    print("Epochs {} train loss {} train acc {} val loss {} val acc {}".format(i, train_loss, train_acc,
                                                                              val_loss, val_acc))
    
    scheduler.step()

In [ ]:
epochs = list(range(1, EPOCHS+1))
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 5))

axes[0].plot(epochs, loss_train)
axes[0].plot(epochs, loss_val)
axes[0].legend(["Training", "Validation"])
axes[0].set_title("Performance: Loss")

axes[1].plot(epochs, acc_train)
axes[1].plot(epochs, acc_val)
axes[1].legend(["Training", "Validation"])
axes[1].set_title("Performance: Accuracy")
plt.suptitle("ResNet-50 fine tuned {}%".format(round(best_acc*100, 2)))
plt.tight_layout()
plt.show()

In [ ]:
def predict(x):
    image = transform(x)
    image = image.reshape(1, 3, IM_SIZE, IM_SIZE)
    if torch.cuda.is_available():
        image = image.cuda()
        
    best_model.eval()
    with torch.no_grad():
        out = best_model(image)
        
    return out.argmax(1).item()

In [ ]:
pred = []
for i in x_test:
    pred += [predict(i)]

In [ ]:
score = accuracy_score(pred, y_test)
report = classification_report(pred, y_test)
print("Accuracy score: ", score)
print("Report: ", report)
sns.heatmap(confusion_matrix(pred, y_test), annot=True)
plt.title("Accuracy {}%".format(round(score*100, 2)))
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(10, 10))
index = 0
for i in range(5):
    for j in range(5):
        axes[i][j].imshow(x_test[index])
        axes[i][j].set_title("Predicted: {}\nReal: {}".format(index_class[pred[index]],
                                                   index_class[y_test[index]]))
        
        index += 1
        
plt.tight_layout()
plt.show()